**Model Health Dashboard**


This dashboard reports statistics about the health of the collection of Drawdown solution models.

In [1]:
import importlib
import os.path

import bqplot
import IPython.display
import ipywidgets
import pandas as pd

import solution.factory
from tools.health import charts

all_solutions = pd.read_csv(os.path.join('data', 'overview', 'solutions.csv'), index_col=False,
                            skipinitialspace=True, header=0, skip_blank_lines=True, comment='#')
total_solutions = len(all_solutions.index)
py_solutions_scenarios = solution.factory.all_solutions_scenarios()

num_py_solutions = len(list(py_solutions_scenarios.keys()))
num_xl_solutions = total_solutions - num_py_solutions

pds_adoption_basis_counts = {
    'Linear': 0,
    'Existing Adoption Prognostications': 0,
    'Bass Diffusion S-Curve': 0,
    'Logistic S-Curve': 0,
    'Fully Customized PDS': 0,
    'Customized S-Curve Adoption': 0,
}
ref_adoption_basis_counts = {
    'Default': 0,
    'Custom': 0,
}
scenarios_per_solution = []
for name in py_solutions_scenarios.keys():
    m = importlib.import_module('solution.'+name)
    scenarios_per_solution.append(len(m.scenarios))
    for scenario in m.scenarios.values():
        pds_adoption_basis_counts[scenario.soln_pds_adoption_basis] += 1
        ref_basis = scenario.soln_ref_adoption_basis
        ref_basis = ref_basis if ref_basis is not None else 'Default'
        ref_adoption_basis_counts[ref_basis] += 1

soln_count_chart = bqplot.Pie(sizes=[num_xl_solutions, num_py_solutions], labels=['Excel only', 'Python & Excel'],
                              display_values=True, values_format='d', display_labels='inside', radius=110,
                              start_angle=60, end_angle=420, colors=['#6495ED', '#fcea3f'])
soln_count_fig = bqplot.Figure(marks=[soln_count_chart], title='Solution implementation')
soln_count_fig.layout.width = '300px'
soln_count_fig.fig_margin = {'left':1, 'right':1, 'top':1, 'bottom':1}
pds_adoption_chart = bqplot.Pie(sizes=list(pds_adoption_basis_counts.values()),
                                labels=list(pds_adoption_basis_counts.keys()),
                                display_labels='outside', radius=140,
                                sort=False, display_values=True, values_format='d')
pds_adoption_fig = bqplot.Figure(marks=[pds_adoption_chart], title='PDS Adoption Basis')
pds_adoption_fig.layout.width = '800px'
pds_adoption_fig.fig_margin = {'left':1, 'right':1, 'top':1, 'bottom':1}

ref_adoption_chart = bqplot.Pie(sizes=list(ref_adoption_basis_counts.values()),
                                labels=list(ref_adoption_basis_counts.keys()),
                                display_labels='inside', radius=110,
                                display_values=True, values_format='d')
ref_adoption_fig = bqplot.Figure(marks=[ref_adoption_chart], title='REF Adoption Basis')
ref_adoption_fig.layout.width = '300px'
ref_adoption_fig.fig_margin = {'left':1, 'right':1, 'top':1, 'bottom':1}


# ------------------- Scenarios per solution -------------------


y_scale = bqplot.LinearScale()
y_axis = bqplot.Axis(scale=y_scale, orientation='vertical', label='solutions')
x_scale = bqplot.LinearScale()
x_axis = bqplot.Axis(scale=x_scale, tick_format='d', label='num scenarios')
hist = bqplot.Hist(sample=scenarios_per_solution, scales={'sample': x_scale, 'count': y_scale})
scenarios_per_solution_chart = bqplot.Figure(marks=[hist], axes=[x_axis, y_axis], padding_y=0,
                                             title="Num scenarios per solution")
scenarios_per_solution_chart.layout.width = '50%'
scenarios_per_solution_chart.layout.height = '300px'


# ------------------- Regional Data -------------------


surveydata = pd.read_csv(os.path.join('data', 'health', 'survey.csv'), index_col=False,
                            skipinitialspace=True, header=0, skip_blank_lines=True, comment='#')
num_scenarios = len(surveydata.index)
regional_nonzero_adoption = surveydata.loc[surveydata['RegionalFractionAdoption'] != 0.0]
regional_nonzero_tam = surveydata.loc[surveydata['RegionalFractionTAM'] != 0.0]

nonzero_count = regional_nonzero_tam.shape[0]
zero_count = num_scenarios - nonzero_count
nonzero_chart = bqplot.Pie(sizes=[nonzero_count, zero_count], labels=['YES', 'NO'], colors=['Green', 'Red'],
                           display_values=True, values_format='d', display_labels='inside', radius=110)
tam_regional_nonzero_fig = bqplot.Figure(marks=[nonzero_chart], title='Has Regional TAM Data?')
tam_regional_nonzero_fig.layout.width = '320px'
tam_regional_nonzero_fig.fig_margin = {'left':1, 'right':1, 'top':1, 'bottom':1}

y_scale = bqplot.LinearScale()
y_axis = bqplot.Axis(scale=y_scale, orientation='vertical', label='# scenarios')
x_scale = bqplot.LinearScale()
x_axis = bqplot.Axis(scale=x_scale, tick_format='.1f', label='percentage')
hist_data = regional_nonzero_tam['RegionalFractionTAM'] * 100.0
hist = bqplot.Hist(sample=hist_data, scales={'sample': x_scale, 'count': y_scale})
hist.bins = 50
tam_regional_chart = bqplot.Figure(marks=[hist], axes=[x_axis, y_axis], padding_y=0,
                                  title="Regional TAM as a % of World")
tam_regional_chart.layout.width = '100%'

nonzero_count = regional_nonzero_adoption.shape[0]
zero_count = num_scenarios - nonzero_count
nonzero_chart = bqplot.Pie(sizes=[nonzero_count, zero_count], labels=['YES', 'NO'], colors=['Green', 'Red'],
                           display_values=True, values_format='d', display_labels='inside', radius=110)
ad_regional_nonzero_fig = bqplot.Figure(marks=[nonzero_chart], title='Has Regional Adoption Data?')
ad_regional_nonzero_fig.layout.width = '320px'
ad_regional_nonzero_fig.fig_margin = {'left':1, 'right':1, 'top':1, 'bottom':1}

y_scale = bqplot.LinearScale()
y_axis = bqplot.Axis(scale=y_scale, orientation='vertical', label='# scenarios')
x_scale = bqplot.LinearScale()
x_axis = bqplot.Axis(scale=x_scale, tick_format='.1f', label='percentage')
hist_data = regional_nonzero_adoption['RegionalFractionAdoption'] * 100.0
hist = bqplot.Hist(sample=hist_data, scales={'sample': x_scale, 'count': y_scale})
hist.bins = 100
ad_regional_chart = bqplot.Figure(marks=[hist], axes=[x_axis, y_axis], padding_y=0,
                                  title="Regional Adoption as a % of World")
ad_regional_chart.layout.width = '100%'


y_scale = bqplot.LinearScale()
y_axis = bqplot.Axis(scale=y_scale, orientation='vertical', label='# scenarios')
x_scale = bqplot.LinearScale()
x_axis = bqplot.Axis(scale=x_scale, tick_format='.1f', label=u'R\u00B2 value')
hist_data = surveydata['Rvalue'].dropna() ** 2
hist = bqplot.Hist(sample=hist_data, scales={'sample': x_scale, 'count': y_scale})
hist.bins = 200
r2_value_chart = bqplot.Figure(marks=[hist], axes=[x_axis, y_axis], padding_y=0,
                                  title="Linearity of PDS Adoption")
r2_value_chart.layout.width = '100%'

ipywidgets.VBox([
    ipywidgets.HBox([soln_count_fig, pds_adoption_fig, ref_adoption_fig]),
    scenarios_per_solution_chart,
    ipywidgets.HBox([tam_regional_nonzero_fig, tam_regional_chart]),
    ipywidgets.HBox([ad_regional_nonzero_fig, ad_regional_chart]),
    r2_value_chart,
])



In [2]:
# ------------------- Land Solution Analytics -------------------

ls_df = pd.read_csv(os.path.join('data', 'health', 'landsurvey.csv'), index_col=0)

# Comparison charts ---------------------------------------------
# Note: assuming most agressive scenario of each solution
perc_tla_comp = charts.soln_comparison(ls_df['% tla'].to_frame(),
                                       title='% of land allocation reached')
perc_world_alloc_comp = charts.soln_comparison(ls_df['% world alloc'].to_frame(),
                                       title='% of World land allocated to solution')
abatement_cost_comp = charts.soln_comparison(ls_df['avg abatement cost'].to_frame(),
                                             title='Average abatement cost ($/tCO2)')

# Has regions ---------------------------------------------
has_regions = ls_df['has regional data']
has_reg_pie = bqplot.Pie(sizes=[len(has_regions[has_regions]), len(has_regions[has_regions == False])],
                         labels=['YES', 'NO'], colors=['Green', 'Red'], display_values=True, values_format='d',
                         display_labels='inside', radius=120)
land_has_reg = bqplot.Figure(marks=[has_reg_pie], title='Has regional adoption data? (Land solutions)')
land_has_reg.fig_margin = {'left':1, 'right':1, 'top':1, 'bottom':1}

# Regional issues ---------------------------------------------
no_issues = exceeds_limits = regions_mismatch = both_issues = 0
for i, row in ls_df[has_regions].iterrows():
    limits_flag = mismatch_flag = False
    if row['ca scen regions exceed world count'] > 0 or row['ca scen world exceeds regions count'] > 0:
        mismatch_flag = True
    if row['ca scen exceeds alloc count'] > 0: limits_flag = True
    if mismatch_flag and limits_flag: both_issues += 1
    elif mismatch_flag: regions_mismatch += 1
    elif limits_flag: exceeds_limits += 1
    else: no_issues += 1
        
reg_issues_pie = bqplot.Pie(sizes=[no_issues, exceeds_limits, regions_mismatch, both_issues],
                            labels=['No issues', 'Exceeds limits', 'Regions-world mismatch', 'Both issues'],
                            display_values=True, values_format='d', display_labels='outside', radius=150)
reg_issues = bqplot.Figure(marks=[reg_issues_pie], title='Issues with regional data in Custom Adoption scenarios')
reg_issues.layout.width = '800px'
reg_issues.fig_margin = {'left':1, 'right':1, 'top':1, 'bottom':1}

ipywidgets.VBox([
    ipywidgets.HBox([perc_tla_comp, perc_world_alloc_comp]),
    ipywidgets.HBox([abatement_cost_comp]),
    ipywidgets.HBox([land_has_reg, reg_issues])
])

In [38]:
# ------------------- Analysis of Scenarios -------------------

scn_df = pd.read_csv(os.path.join('data', 'health', 'scenario_uniq_values.csv'), index_col=0)
scn_cnt_df = scn_df.apply(pd.Series.value_counts).fillna(0.0).T
scn_cnt_df['solutions'] = scn_cnt_df.sum(axis=1)  - scn_cnt_df[1.0]
scn_cnt_df = scn_cnt_df.drop(scn_cnt_df[scn_cnt_df.solutions == 0.0].index)
# delete some rows which are not interesting
scn_cnt_df = scn_cnt_df.drop(axis=0, labels=['soln_ref_adoption_basis', 'soln_ref_adoption_custom_name',
    'soln_pds_adoption_basis', 'soln_pds_adoption_custom_name', 'soln_pds_adoption_prognostication_source',
    'soln_pds_adoption_prognostication_trend', 'soln_pds_adoption_prognostication_growth', 'pds_source_post_2014',
    'pds_adoption_use_ref_years', 'pds_adoption_final_percentage', 'use_custom_tla'])

charts.soln_comparison(scn_cnt_df[['solutions']], title='parameters which differ between scenarios within solution')

Figure(axes=[Axis(grid_lines='none', orientation='vertical', scale=OrdinalScale()), Axis(label='solutions', sc…